# Advanced scikit-learn

Authors: [Alexandre Gramfort](http://alexandre.gramfort.net), [Thomas Moreau](https://tommoral.github.io/about.html), and [Pedro L. C. Rodrigues](https://plcrodrigues.github.io/).

<img src="figures/sklearn-logo.png" width="150">

The aim of this notebook is:

  - To explain the **full scikit-learn API** (estimators, transformers, classifiers, regressors, splitters). 
      - A good reference is [here](https://scikit-learn.org/1.5/developers/develop.html) too.
  - To explain how to assemble these objects in complex **pipelines with mixed data types** (numerical, categorical, etc.) using `Pipeline` and `ColumnTransformer` objects.
  - Have you **write your own** transformer, splitter and classifier.
  
## Table of contents

* [1 Working only with numerical data](#workingnumerical)
    * [1.1 Pandas preprocessing](#workingnumerical_pandas)
    * [1.2 Making it less error prone using scikit-learn](#workingnumerical_errorprone)    
* [2 Working only with categorical data](#workingcategorical)
* [3 Combining both categorical and numerical data in the pipeline](#combining)
* [4 Automated pipeline with skrub](#skrub)
* [5 From one split to cross-validation](#crossvalidation)

To explain these concepts we will start from a full working code based on the Titanic dataset. 

Then, we will deconstruct all the blocks and start writing our own Python classes.

First, let's fetch the Titanic dataset directly from [OpenML](https://openml.org/).

In [ ]:
import pandas as pd
from sklearn.datasets import fetch_openml

In [ ]:
X_df, y = fetch_openml("titanic", version=1, as_frame=True, return_X_y=True)
X_df.head()

The classification task is to predict whether or not a person will survive the Titanic disaster.

In [ ]:
y

We will split the data into a training and a testing set.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_df, y, random_state=42
)

In [ ]:
train_test_split?

<div class="alert alert-success">
    <p><b>QUESTIONS</b>:</p>
    <ul>
        <li>What would happen if you tried to fit a <tt>LogisticRegression</tt> classifier?</li>
    </ul>
</div>

# 1 Working only with numerical data <a class="anchor" id="workingnumerical"></a> [↑](#Table-of-contents)

Let's start with a model using only numerical columns.

In [ ]:
X_df.dtypes

## 1.1 Pandas preprocessing  <a class="anchor" id="workingnumerical_pandas"></a> [↑](#Table-of-contents)

Before using scikit-learn, we will do some simple preprocessing using pandas. 

First, let's select only a few of the numerical columns:

In [ ]:
num_cols = ['pclass', 'age', 'parch', 'fare']

X_train_num = X_train[num_cols]
X_test_num = X_test[num_cols]

<div class="alert alert-success">
    <p><b>QUESTIONS</b>:</p>
    <ul>
        <li>And now, what would happen if you tried to fit a <tt>LogisticRegression</tt> classifier ?</li>
    </ul>
</div>

We might want to look into a summary of the data that we try to fit.

In [ ]:
X_train_num.info()

Since there are some missing data, we can replace them with a mean.

In [ ]:
X_train_num_imputed = X_train_num.fillna(X_train_num.mean())
X_train_num_imputed.info()

In [ ]:
model.fit(X_train_num_imputed, y_train)

<div class="alert alert-success">
    <p><b>EXERCISE</b>:</p>
    <ul>
    <li>What should we do if there are also missing values in the test set?</li>
    <li>Process the test set so as to be able to compute the test score of the model.</li>
    </ul>
</div>

Solution is in `solutions/01-pandas_fillna_test.py`

See: https://scikit-learn.org/stable/common_pitfalls.html

## 1.2 Making it less error prone using scikit-learn <a class="anchor" id="workingnumerical_errorprone"></a> [↑](#Table-of-contents)

Scikit-learn provides several useful **transformers** to preprocess data. Its basic API is illustrated below:

<img src="figures/sklearn-transformers-api.png" width="450">

Basically, a transformer is used in three sequential steps:

1. Fit the transformer to the training data
2. Transform the training data
3. Transform the test data

Some examples of transformers are [scalers](https://scikit-learn.org/1.5/modules/preprocessing.html#standardization-or-mean-removal-and-variance-scaling), [normalizers](https://scikit-learn.org/1.5/modules/preprocessing.html#normalization), and [categorical encoders](https://scikit-learn.org/1.5/modules/preprocessing.html#encoding-categorical-features). 

The `sklearn.impute.SimpleImputer` is a transformer allowing for the same job than the processing done with Pandas above. As we will see later, it integrates nicely with other scikit-learn components when using pipelines.

In [ ]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="mean")

In [ ]:
SimpleImputer?

As any estimator in scikit-learn, a transformer has a `fit` method which should be called on the training data to learn the required statistics. In the case of a mean imputer, we need to compute the mean for each feature.

In [ ]:
imputer.fit(X_train_num)

In [ ]:
imputer.statistics_

To impute the values by the mean, we can use the `transform` method.

In [ ]:
imputer.transform(X_train_num)

As previoulsy mentioned, we should always use the values estimated using the training set in `fit` when imputing the test set. Otherwise, we risk suffering from overfitting due to [data leakage](https://scikit-learn.org/stable/common_pitfalls.html#data-leakage).

<div class="alert alert-warning">
<b>What is a "Transformer"?</b>: <br/>

Every transform in `sklearn` implements at least the following methods:

<ul>
    <li><tt>fit(X, y=None)</tt></li>
    <li><tt>transform(X)</tt></li>
    <li><tt>get_params()</tt></li>
    <li><tt>set_params(**kwargs)</tt></li>  
</ul>
</div>

In [ ]:
params = imputer.get_params()
params

In [ ]:
imputer.fit?

In [ ]:
imputer.transform?

Let's look at the attributes of our `imputer`

In [ ]:
public_attributes = [attr for attr in dir(imputer) if not attr.startswith('_')]
public_attributes

We have among these attributes:

- **parameters** (keys in get_params method output)
- **methods** (fit, transform, etc.)
- **estimated quantities** that appear after a `fit` (ending with `_`)

In [ ]:
public_methods = [
    attr for attr in dir(imputer)
    if not attr.startswith('_') and
    not attr.endswith('_') and
    attr not in params]
public_methods

In [ ]:
imputer.inverse_transform?

Estimated quantities:

In [ ]:
fit_attributes = [
    attr for attr in dir(imputer)
    if not attr.startswith('_') and
    attr.endswith('_')]
fit_attributes

<div class="alert alert-success">
    <p><b>EXERCISE</b>:</p>
    <ul>
        <li>What are the attributes of a <tt>LogisticRegression</tt> classifier?. You will decompose these in the 3 categories.</li>
    </ul>
</div>

### Using a Pipeline

We saw earlier that we should be careful when preprocessing data to avoid any "data leak" (i.e. reusing some knowledge from the training when testing our model). Scikit-learn provides the `Pipeline` class to make successive transformations. In addition, it will ensure that the right operations will be applied at the right time.

In [ ]:
from sklearn import set_config

set_config(display='diagram')

In [ ]:
from sklearn.pipeline import make_pipeline

model = make_pipeline(SimpleImputer(strategy='mean'),
                      LogisticRegression())
model.fit(X_train_num, y_train)

In [ ]:
model.steps

Alternative syntax using named "steps".

In [ ]:
from sklearn.pipeline import Pipeline

model = Pipeline([
    ("imputer", SimpleImputer(strategy='mean')),
    ("classifier", LogisticRegression())    
])
model.fit(X_train_num, y_train)

In [ ]:
model.score(X_test_num, y_test)

In [ ]:
model.steps

Saving your estimator in HTML for presentations, blog posts etc.

In [ ]:
from sklearn.utils import estimator_html_repr

with open('model.html', 'w') as fid:
    fid.write(estimator_html_repr(model))

# !open model.html

### Manipulating Pipeline steps

A pipeline is a sequence of `steps`. Each `step` is a scikit-learn estimator. All steps except the last one are typically **transformers** (i.e they have `fit`, `fit_transform`, `transform` methods) and the last step is a **classifier** or a **regressor**.

In [ ]:
model.steps  # accessing steps as a list

In [ ]:
model.named_steps  # accessing steps with their names as a dict

In [ ]:
model[:1]  # slicing a pipeline

In [ ]:
model[-1]

In [ ]:
from sklearn.base import is_classifier

is_classifier(model[-1])

For comparison, we can decompose the pipeline and chain the operations manually, mimicking what the `Pipeline` object does internally:

In [ ]:
preprocessor = model[:-1]
classifier = model[-1]

X_train_preproc = preprocessor.fit_transform(X_train_num, y_train)
X_test_preproc = preprocessor.transform(X_test_num)

classifier.fit(X_train_preproc, y_train)
classifier.score(X_test_preproc, y_test)

### Towards a ColumnTransformer

If we want to avoid creating `X_train_num` and fit the model on `X_train`, we can select the numerical columns using a `ColumnTransformer` object:

In [ ]:
from sklearn.compose import ColumnTransformer

ColumnTransformer?

In [ ]:
numerical_preprocessing = ColumnTransformer([
    ("imputer", SimpleImputer(strategy='mean'), num_cols)
])
    
model = Pipeline([
    ("numerical preproc.", numerical_preprocessing),
    ("classifier", LogisticRegression()),
])

model.fit(X_train, y_train)

In [ ]:
num_cols

In [ ]:
model.score(X_test, y_test)

# 2 Working only with categorical data <a class="anchor" id="workingcategorical"></a> [↑](#Table-of-contents)

Categorical columns are not supported natively by machine-learning algorithms and require a preprocessing step commonly called encoding. The most classical categorical encoders are the `OrdinalEncoder` and the `OneHotEncoder`, but many others exist and are implemented in `sklearn`; see [here](https://scikit-learn.org/dev/modules/preprocessing.html#encoding-categorical-features). 

Let's see what happens if we try to classify the data using only categorical features encoded via `OrdinalEncoder`.

In [ ]:
X_train.head()

In [ ]:
cat_cols = ['sex', 'embarked', 'pclass']

In [ ]:
X_train_cat = X_train[cat_cols]

In [ ]:
X_train_cat.info()

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

cat_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy='constant', fill_value='missing')),
    ("ordinal_encoder", OrdinalEncoder())
])

categorical_preprocessing = ColumnTransformer([
    ("categorical_preproc", cat_pipeline, cat_cols)
])

model = Pipeline([
    ("categorical_preproc", categorical_preprocessing),
    ("classifier", LogisticRegression())
])
model.fit(X_train, y_train)

In [ ]:
model.score(X_test, y_test)

Remember that `Pipeline` is yet another scikit-learn estimator. It therefore has the methods `set_params` and `get_params` that can be inspected:

In [ ]:
model.get_params()

In [ ]:
model.set_params(classifier__C=0.1)
model.get_params()

<div class="alert alert-success">
    <p><b>EXERCISE</b>:</p>
    There exists, of course, many other strategies to encode categorical variables. In this exercise, you will write one by yourself that we shall call <tt>CountEncoder</tt>. The idea is to replace categorical variables with their count in the train set. When you've finished coding, change the ordinal encoder in the previous pipeline with an instance of your <tt>CountEncoder</tt> and see how the classification performance changes.
</div>

Your class will needs to inherit from `BaseEstimator`, `TransformerMixin` in `sklearn.base` submodule.

You will use the class `Counter` from the `collections` module in the Python standard library.

You will apply your code on this toy example

```python
>>> X = np.array([
...    [0, 2],
...    [1, 3],
...    [1, 1],
...    [1, 1],
... ])
>>> ce = CountEncoder()
>>> print(ce.fit_transform(X))
[[1 1]
 [3 1]
 [3 2]
 [3 2]]
```

Solution is in `solutions/01-count_encoder.py`.

# 3 Mix categorical and numerical data in a pipeline <a class="anchor" id="combining"></a> [↑](#Table-of-contents)

<div class="alert alert-success">
    <p><b>EXERCISE</b>:</p>
    <ul>
    <li>Try to combine the numerical and categorical pipelines into a single <tt>ColumnTransformer</tt></li>
        <li>Fit a <tt>LogisticRegression</tt> classifier on the output of this feature engineering. How does the test score evolve?</li>
    </ul>
</div>

Solution is in `solutions/01b-full_column_transformer.py`

# 4 An automated pipeline with Skrub <a class="anchor" id="skrub"></a> [↑](#Table-of-contents)

The procedure carried out so far was, until very recently, the standard way of handling tabular data mixing categorical and numerical values. Although this works well, it is not very straightforward and can require a great deal of expertise from the user. This was one of the main motivations behind the creation of [`skrub`](https://skrub-data.org/stable/index.html), a Python library that facilitates the preprocessing and feature engineering of tabular machine learning.

<img src="figures/skrub-logo.png" width="140">

One of the most important objects in `skrub` is the `TableVectorizer`, which automatically chooses which encoding strategy should be used for each column depending on its cardinality. See below what we get when using it as the first step in a `sklearn` pipeline:

In [ ]:
# !pip install -U skrub

In [ ]:
from skrub import TableVectorizer

model = Pipeline([
    ("table_vectorizer", TableVectorizer()),
    ("imputer", SimpleImputer(strategy='mean')),
    ("classifier", LogisticRegression(solver='liblinear'))
])
model.fit(X_train, y_train)

In [ ]:
print(model.score(X_test, y_test))

# 5 From one split to cross-validation <a class="anchor" id="crossvalidation"></a> [↑](#Table-of-contents)

Cross-validation objects are parametrized to split data in multiple train and test splits as described [here](https://scikit-learn.org/1.5/modules/cross_validation.html).

These objects are called **splitters** and consist of

- A `split` method that generate indices to partition data into training and test set.
- A `get_n_splits` method that returns the number of splitting iterations in the cross-validator

See [this](https://github.com/scikit-learn/scikit-learn/blob/6e9039160f0dfc3153643143af4cfdca941d2045/sklearn/model_selection/_split.py#L109) chunk of code for the structure of a basic splitter.

Given a `model`, some data `X, y` and a splitter one can fit and score on
all requested data splits. Some functions to do this automatically are `cross_val_score`
(historical way) and `cross_validate` (more modern way).

Let's first define a model:

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

cat_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("ordinal_encoder", OrdinalEncoder())
])

categorical_preprocessing = ColumnTransformer([
    ("categorical_preproc", cat_pipeline, cat_cols)
])

model = Pipeline([
    ("categorical_preproc", categorical_preprocessing),
    ("classifier", LogisticRegression())
])
model

Let's now use a default 5-fold CV. You will see that there is a large amount of discrepancy among the test_score values

In [ ]:
from sklearn.model_selection import cross_validate

cv_results = cross_validate(model, X_df, y, cv=5)
pd.DataFrame(cv_results)

In [ ]:
cross_validate?

In [ ]:
pd.DataFrame(cv_results).agg(['mean', 'std'])

The reason is that default CV object (here 5-fold CV) is deterministic, while the distribution of "survivor" is not uniform in the dataset. See:

In [ ]:
y.astype(int).to_frame().groupby(y.index.values // 100).mean().plot()

To fix this one needs stratified folds (so that the fraction of "survivors" is the same in each fold) but also to shuffle the data.

In [ ]:
from sklearn.model_selection import StratifiedKFold

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_results = cross_validate(model, X_df, y, cv=cv)
pd.DataFrame(cv_results)

In [ ]:
pd.DataFrame(cv_results).agg(['mean', 'std'])

The variance across folds is now much smaller which is great!

Let's look at the cross-validation scheme with a pretty plot:

In [ ]:
from plotting_utils import plot_cv_indices
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(9, 4))

for shuffle, ax in zip([False, True], axes):
    cv = StratifiedKFold(n_splits=5, shuffle=shuffle)
    plot_cv_indices(cv, X_df, y, ax=ax)

fig.tight_layout()

See [this page](https://scikit-learn.org/stable/auto_examples/model_selection/plot_cv_indices.html) for more details and a list of the different cross-validation objects.

### Writing your own cross-validation object

As mentioned above, every splitter should implement a `split` and `get_n_splits` method. 

- `split` should return an iterable of tuples of indices. This is isually done using the concept of `yield` in Python for generators.
- `get_n_splits` returns an integer corresponding to the number of splits/folds chosen at the initialization of the splitter.

Let's first see what we get with the `cv` object defined before.

In [ ]:
cv.get_n_splits()

In [ ]:
for train_idx, test_idx in cv.split(X_df, y):
    print(train_idx[:5], test_idx[:5])
    print(f"N. samples train: {len(train_idx)}  -- N. samples test: {len(test_idx)}")

<div class="alert alert-success">
    <p><b>EXERCISE</b>:</p>
    <ul>
        <li>Imagine that the index of <code>y</code> gives you some provenance about the sample (e.g. which cohort of subjects in a clinical study). Write a splitter that allows to test the performance of a model on a left-out cohort. In other words, you will do as many splits as the number of unique values in <code>y.index.values</code>, and predict of each left-out cohort. To simulate this, we will modify the index variable <code>y</code>, just for educational purposes.
        </li>
    </ul>
</div>

Solution is in `solutions/01c-splitter.py`

### When you're done with this notebook you can do the assignments on scikit-learn.